In [18]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

import urllib

print('Libraries imported.')


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         239 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0         conda-forge
    geopy:         1.20.0-py_0       conda-forge

The following p

In [19]:
page = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

toronto_html = BeautifulSoup(page, 'html.parser')

table_html = toronto_html.body.table.extract()

In [20]:
table = []
for x in table_html.find_all("td"):
    table.append(x.get_text())

table_zip = []
table_borough = []
table_neighborhood = []

for x in range(int(len(table)/3)):
    table_zip.append(table[x*3])

for x in range(int(len(table)/3)):
    table_borough.append(table[x*3+1])
    
for x in range(int(len(table)/3)):
    neighborhood = table[x*3+2]
    table_neighborhood.append(neighborhood[:-1])

In [21]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
neighborhoods = pd.DataFrame(columns = column_names)
neighborhoods

,PostalCode,Borough,Neighborhood


In [22]:
for x in range(int(len(table)/3)):
    neighborhoods = neighborhoods.append({'PostalCode': table_zip[x], 'Borough': table_borough[x], 'Neighborhood': table_neighborhood[x]}, ignore_index = True)
neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [23]:
neighborhoods = neighborhoods[neighborhoods.Borough != 'Not assigned']
neighborhoods = neighborhoods.reset_index(drop = True)
neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [24]:
for x in range(len(neighborhoods)):
    if neighborhoods.Neighborhood[x] == 'Not assigned':
        neighborhoods.Neighborhood[x] = neighborhoods.Borough[x]

neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [25]:
for x in range(len(neighborhoods)-1):
    if neighborhoods.PostalCode[x] == neighborhoods.PostalCode[x+1]:
        neighborhoods.Neighborhood[x+1] = neighborhoods.Neighborhood[x] + ", " + neighborhoods.Neighborhood[x+1]

In [28]:
length = len(neighborhoods) - 1
neighborhood = neighborhoods
for x in range(length):
    if neighborhoods.PostalCode[x] == neighborhoods.PostalCode[x+1]:
        neighborhood = neighborhood.drop(neighborhoods.index[x])
neighborhood = neighborhood.reset_index(drop = True)
neighborhood.head(11)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [27]:
neighborhood.shape

(103, 3)